In [1]:
import os
import pandas as pd
import glob

### Streamflow quality data processing

In [3]:
# Data directory
streamflow_directory = '//gmvstorage.gmv.es/storage/anin/Streamflow/All_streamflow_data'  # CSV directory path
fNs = glob.glob(f'{streamflow_directory}/*.csv')

In [4]:
# Initialize variables
min_date = None
max_date = None

# Dictionary to store the data from each file
data = {}

In [5]:
# Find the common time range and collect data
for file in fNs:
    df = pd.read_csv(file, parse_dates=[0], dayfirst=True, index_col=0, skiprows=3)
    dates = df.index
    
    file_min_date = dates.min()
    file_max_date = dates.max()
    
    if min_date is None or file_min_date < min_date:
        min_date = file_min_date
    if max_date is None or file_max_date > max_date:
        max_date = file_max_date
    
    # Store the data in the dictionary
    column_name = os.path.basename(file)[:-4] # Using the filename as the column name
    data[column_name] = df.iloc[:,1] # Quality data is stored on the second column of each csv file

In [6]:
# Create consolidated DataFrame
date_range = pd.date_range(start=min_date, end=max_date)
consolidated_df = pd.DataFrame(index=date_range)

# Merge data from each file into the consolidated DataFrame
for file_name, df in data.items():
    consolidated_df[file_name] = df # Assign the data to the corresponding column
# Resample the DataFrame to monthly frequency
monthly_df = consolidated_df.resample('M').mean()

C:\Users\ecig\AppData\Local\Temp\ipykernel_20068\3256307030.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  consolidated_df[file_name] = df # Assign the data to the corresponding column
C:\Users\ecig\AppData\Local\Temp\ipykernel_20068\3256307030.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  consolidated_df[file_name] = df # Assign the data to the corresponding column
C:\Users\ecig\AppData\Local\Temp\ipykernel_20068\3256307030.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

In [7]:
# Iterate through the column names and remove "_140" from the ones that end with it
for column in consolidated_df.columns:
    if column.endswith('_140'):
        new_column = column[:-4]  # Remove the last 4 characters ("_140")
        monthly_df.rename(columns={column: new_column}, inplace=True)
print(monthly_df)

            C5H003  C5H007  C5H012  C5H014  C5H022  C5H035  C5H048  C5H053  \
1979-10-31   170.0     1.0    60.0   170.0   255.0   255.0   255.0   255.0   
1979-11-30   170.0     1.0     1.0   170.0   255.0   255.0   255.0   255.0   
1979-12-31   170.0     1.0     1.0   170.0   255.0   255.0   255.0   255.0   
1980-01-31   170.0     1.0     1.0   170.0   255.0   255.0   255.0   255.0   
1980-02-29   170.0     1.0     1.0   170.0   255.0   255.0   255.0   255.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
2023-06-30   255.0   255.0   255.0   255.0   255.0   255.0   255.0   255.0   
2023-07-31   255.0   255.0   255.0   255.0   255.0   255.0   255.0   255.0   
2023-08-31   255.0   255.0   255.0   255.0   255.0   255.0   255.0   255.0   
2023-09-30   255.0   255.0   255.0   255.0   255.0   255.0   255.0   255.0   
2023-10-31   255.0   255.0   255.0   255.0   255.0   255.0   255.0   255.0   

            C5H054  C5H056  ...  B8H009  B8H010  B8H014  B8H017

In [8]:
# Export the consolidated DataFrame to an Excel file
monthly_df.to_excel('Streamflow_quality.xlsx', index=True)